In [ ]:
!pip install sagemaker
## Note: you might need to restart the session

## Create sagemaker boto3 session
Replace values with your credentials. The credentials need SageMaker access.

In [ ]:
import boto3

boto_session = boto3.session.Session(
                        aws_access_key_id = "XXXXXXXXXXXXXXXXXXXXXX",
                        aws_secret_access_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                        region_name = "us-east-2"
                        )

## List IAM roles (Optional)

In [ ]:
roles = boto_session.client("iam").list_roles()
Role_list = roles['Roles']
for key in Role_list:
    print(key['RoleName'] + " - " + key['Arn'])

## Find the ModelPackage ARN in the AWS Marketplace, and the execution role

Find the ModelPackage ARN:
 

*   Browse to https://aws.amazon.com/marketplace/pp/prodview-mc7fhbqx7ngy6
*   Subscribe to the ModelPackage if needed
*   Click on Configuration
*   Select a region (ex us-east-1)
*   Write down the product ARN (ex arn:aws:sagemaker:us-east-2:XXXXXXXXXX:model-package/sigmodata-sentiment-X-XXXXXXXXXXXXXXXXXXXXX)




In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

sagemaker_session = sage.Session(boto_session=boto_session)

#@markdown Enter a role or role ARN with Sagemaker permissions
role = "my-sagemaker-iam-role" #@param {type:"string"}
print(sagemaker_session.expand_role(role))

#@markdown Enter the product ARN ex arn:aws:sagemaker:us-east-2:XXXXXXXXXX:model-package/sigmodata-sentiment-X-XXXXXXXXXXXXXXXXXXXXX
modelpackage_arn = "arn:aws:sagemaker:us-east-2:XXXXXXXXXX:model-package/sigmodata-sentiment-X-XXXXXXXXXXXXXXXXXXXXX" #@param {type:"string"}


## Prepare a model package

In [ ]:
from sagemaker import ModelPackage
from sagemaker.predictor import csv_serializer

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, serializer=csv_serializer)

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

## Prepare csv input data

In [ ]:
import pandas as pd

input_data = pd.read_csv(pd.compat.StringIO("""
I feel pretty bad about this right now
The weather is great today
This movie was terrible
I enjoyed sleeping in yesterday
"Don't waste your time, this is not worth it"
This was brilliant
"What he did was neither clever nor intelligent"
"I find it charming"
"The pace was slow and repetitive but I enjoyed it"
"I can't help but feel sorry for her"
"""), header=None, names=["input"])

input_data

## Upload the csv to Sagemaker S3

You can also chose to upload it to your own S3 bucket

In [ ]:
TRANSFORM_WORKDIR = "/tmp"
input_data.to_csv(TRANSFORM_WORKDIR + "/batchtransform_test.csv", index=False, header=False)

transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR + "/batchtransform_test.csv")
print("Transform input uploaded to " + transform_input)

In [ ]:
transformer = model.transformer(1, 'ml.m4.xlarge')
transformer.transform(transform_input, content_type='text/csv')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

## Do batch transform

Results are saved to s3 as CSV


In [ ]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], "batchtransform_test.csv")

s3_client = sagemaker_session.boto_session.client('s3')

response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')
print(response_bytes)

## Deploy the model

This creates an online endpoint for live predictions

In [ ]:
predictor = model.deploy(1, 'ml.m4.xlarge')

## Extract sentiment using csv input
Send phrases to be analyzed for sentiment

Sagemaker's default predictor input format is csv. 

We send the phrases separated by \n

The endpoint replies in csv format.

In [ ]:

csv_results = predictor.predict("\n".join(input_data["input"])).decode('utf-8')

pd.read_csv(pd.compat.StringIO(csv_results), header=None, names=["input", "sentiment", "confidence"])


## Use JSON as content-type for input
Create another predictor for the same endpoint, but specify json as the content type.
The endpoint will return data in json format

In [ ]:
json_predictor = sage.predictor.RealTimePredictor(model.endpoint_name, sagemaker_session, content_type="application/json")

import json
result = json_predictor.predict(json.dumps({
    "input": list(input_data["input"])
})
).decode('utf-8')

json.loads(result)


## Cleanup: delete endpoint

In [ ]:
predictor.delete_endpoint()